In [ ]:
skatt = spark.read.path("/skatt/person/rawdata-2019")

In [ ]:
import pyspark.sql.functions as sf
from pyspark.sql.functions import explode
konto = skatt.select("personidentifikator", "konto")\
    .withColumnRenamed("konto", "kontoer")\
    .withColumn("konto", explode("kontoer"))\
    .select("personidentifikator", "konto.*")\


In [ ]:
konto.printSchema()

root
 |-- personidentifikator: string (nullable = true)
 |-- bankensNavn: string (nullable = true)
 |-- organisasjonsnummer: string (nullable = true)
 |-- kontonummer: string (nullable = true)
 |-- erBoligsparekontoIAnnenEOeSStat: boolean (nullable = true)
 |-- innskudd: long (nullable = true)
 |-- opptjenteRenter: long (nullable = true)
 |-- gjeld: long (nullable = true)
 |-- paaloepteRenter: long (nullable = true)
 |-- tilbakefoerteRenter: long (nullable = true)
 |-- betalteMisligholdteRenter: long (nullable = true)
 |-- tapValutalaan: long (nullable = true)
 |-- gevinstValutalaan: long (nullable = true)
 |-- sparebeloepTilBoligsparingForUngdomOppspartISkattleggingsperioden: long (nullable = true)
 |-- landkode: string (nullable = true)
 |-- overfoertFraBarn: boolean (nullable = true)
 |-- andelOverfoert: double (nullable = true)



In [ ]:
%%time
konto.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path("/felles/skatt/konto-flatt")

CPU times: user 37.9 ms, sys: 19.5 ms, total: 57.4 ms
Wall time: 6.47 s


In [ ]:
konto_flatt = spark.read.path("/felles/skatt/konto-flatt")

In [ ]:
%%time
import pyspark.sql.functions as sf
konto_flatt.groupBy("personidentifikator")\
    .agg(\
        sf.sum("innskudd").alias("sum_innskudd")\
    ).agg(sf.sum("sum_innskudd")).show()


+-----------------+
|sum(sum_innskudd)|
+-----------------+
|          1025400|
+-----------------+

CPU times: user 6.94 ms, sys: 3.67 ms, total: 10.6 ms
Wall time: 2.21 s
